<a href="https://colab.research.google.com/github/Eya-Manai/form-/blob/main/Tp_Cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***TP Cuda réalisé par Eya Manai 1ING 1***

---




Objectif :
Se familiariser avec la programmation CUDA ; Comparer un programme C simple d’une boucle « for »
et celui d’une boucle parallélisée avec CUDA.

**📌 Étape 1: Vérifier que le GPU est bien activé**



In [ ]:
!nvidia-smi


Tue Apr  1 18:53:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**📌 Étape 2:L'exécution d'une boucle « for » CPU**

In [ ]:

%%writefile testcpu.c
#include <stdio.h>
#include <time.h>

#define N 10  // Définition d'une constante pour éviter un tableau de taille variable

void increment_cpu(int *a, int size) {
    for (int i = 0; i < size; i++) {
        a[i] += 1;
    }
}

int main(void) {
    int a[N] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};  // Tableau initialisé avec des valeurs fixes

    // Incrémentation sur le CPU

    clock_t start_cpu = clock();
    increment_cpu(a, N);
    clock_t end_cpu = clock();
    double time_cpu = ((double)(end_cpu - start_cpu)) / CLOCKS_PER_SEC * 1000.0;  // en ms


    // Affichage du résultat
    printf("Résultat après exécution sur CPU : ");
    for (int i = 0; i < N; i++) {
        printf("%d ", a[i]);
    }
    printf("\n");

   // Afficher le temps CPU
    printf("Temps d'exécution CPU : %.6f ms\n", time_cpu);

    return 0;
}



Overwriting testcpu.c



**Compiler le code CPU**

In [ ]:
!gcc testcpu.c -o testcpu


**Exécuter le code CPU**

In [ ]:
!./testcpu


Résultat après exécution sur CPU : 1 2 3 4 5 6 7 8 9 10 
Temps d'exécution CPU : 0.001000 ms


📌 **Étape 3:Comparer avec l'éxécution GPU**

In [ ]:
%%writefile testGPU.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <time.h>


#define N 10  // Définition d'une constante

// Définition du kernel
__global__ void increment_gpu(int *a, int size) {
    int i = threadIdx.x;
    if (i < size) {
        a[i] += 1;
    }
}

int main(void) {
    int h_a[N] = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9};  // Tableau initialisé

    // Allocation de la mémoire sur le GPU
    int *d_a;
    cudaMalloc((void **)&d_a, N * sizeof(int));

    // Copie du tableau h_a sur le GPU
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);

    // Mesure du temps GPU

    cudaEvent_t start, stop;
    float time_gpu;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);


    // Lancer le kernel
    increment_gpu<<<1, N>>>(d_a, N);

    // Synchroniser le GPU
    cudaDeviceSynchronize();
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_gpu, start, stop);

    // Copier le résultat du GPU vers le CPU
    cudaMemcpy(h_a, d_a, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Afficher le résultat
    printf("Résultat après exécution sur GPU : ");
    for (int i = 0; i < N; i++) {
        printf("%d ", h_a[i]);
    }
    printf("\n");
    // Afficher le temps GPU
    printf("Temps d'exécution GPU : %.6f ms\n", time_gpu);

    // Libérer la mémoire GPU
    cudaFree(d_a);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Overwriting testGPU.cu


 **Compiler le code GPU **

In [ ]:
!nvcc testGPU.cu -o testGPU


**Exécuter le code CUDA**

In [ ]:
!./testGPU

Résultat après exécution sur GPU : 0 1 2 3 4 5 6 7 8 9 
Temps d'exécution GPU : 9.524832 ms


**Obsrvation des résultats**

Aprés l'exécution des deux codes on remarque :     

*   Le tableau final est identique pour les deux méthodes
*   Le CPU est plus rapide que le GPU


*   Le cout du lancement du kernel sur le GPU est élevé
*   CUDA est plus pratique pour les calculs massivement parallèles mais pas pour les petits calculs avec peu de threads


*   Le GPU est plus performant pour les grandes tailles de données
*   Le CPU execute le programme d'une façon séquentielle

increment_gpu<<<1, N>>>(d_a, N);
Lance un seul bloc (<<<1, N>>>), ce qui signifie qu'il s'exécute sur un seul multiprocesseur de streaming (SM) avec un parallélisme limité. Le processeur, quant à lui, exécute la boucle de manière séquentielle, mais très efficace.











**Generation d'un grand nombre de données**


**CPU**

In [40]:
%%writefile test2cpu.c
#include <stdio.h>
#include <time.h>
#include <stdlib.h>

#define N 1000000  // Définition d'une constante pour éviter un tableau de taille variable

void increment_cpu(int *a, int size) {
    for (int i = 0; i < size; i++) {
        a[i] += 1;
    }
}

int main(void) {

    int a[N];
    for(int i=0;i<10;i++){
      a[i]=rand()%101;
    }
    // Incrémentation sur le CPU

    clock_t start_cpu = clock();
    increment_cpu(a, N);
    clock_t end_cpu = clock();
    double time_cpu = ((double)(end_cpu - start_cpu)) / CLOCKS_PER_SEC * 1000.0;  // en ms


    // Affichage du résultat
    printf("Résultat après exécution sur CPU : ");
    for (int i = 0; i < N; i++) {
        printf("%d ", a[i]);
    }
    printf("\n");

   // Afficher le temps CPU
    printf("Temps d'exécution CPU : %.6f ms\n", time_cpu);

    return 0;
}



Overwriting test2cpu.c


In [41]:
!gcc test2cpu.c -o test2cpu


In [42]:
!./test2cpu


Résultat après exécution sur CPU : 33 33 55 13 53 57 9 31 45 95 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

**GPU**

In [32]:
%%writefile test2GPU.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <time.h>
#include <stdlib.h>

#define N 1000000  // Increase N to make the GPU advantage visible
#define THREADS_PER_BLOCK 256  // Define a reasonable block size


// Noyau GPU optimisé avec boucle Grid-Stried
__global__ void increment_gpu(int *a, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    while (i < size) {
        a[i] += 1;
        i += blockDim.x * gridDim.x;
    }
}

int main(void) {
    int *h_a, *d_a;

    // Allocation et initialisation de la mémoire pour l'hote
    h_a = (int*)malloc(N * sizeof(int));
    srand(time(NULL));
    for(int i = 0; i < N; i++){
        h_a[i] = rand() % 101;  //gerer les élements aléatoirement
    }

    // Allocation memoire gpu
    cudaMalloc((void **)&d_a, N * sizeof(int));

    // Copier data du cpu vers GPU
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);

    // Commencer temps GPU
    cudaEvent_t start, stop;
    float time_gpu;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Configurer le kernel avec pleusieurs blocks
    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    increment_gpu<<<blocks, THREADS_PER_BLOCK>>>(d_a, N);
    cudaDeviceSynchronize();

    // Stop Chrono
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&time_gpu, start, stop);

    // Copier resultats vers CPU
    cudaMemcpy(h_a, d_a, N * sizeof(int), cudaMemcpyDeviceToHost);

    printf("First 10 values after GPU increment: ");
    for (int i = 0; i < 10; i++) {
        printf("%d ", h_a[i]);
    }
    printf("...\n");

    printf("GPU Execution Time: %.6f ms\n", time_gpu);

    // Free memory
    cudaFree(d_a);
    free(h_a);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}



Overwriting test2GPU.cu


In [33]:
!nvcc test2GPU.cu -o test2GPU

In [34]:
!./test2GPU

First 10 values after GPU increment: 43 33 35 91 28 45 73 91 40 39 ...
GPU Execution Time: 7.782624 ms


**Obsrvations**

Losque N augmante :

*  Le cpu devient lent car il exécute le calcul séquentiel
*  Le GPU commence à etre plus perforament grace à son architecture massivement parallèle.


*   Les transferts cpu Gpu devient un facteur clé
*   👉 Le GPU est la seule solution viable pour traiter de très grands ensembles de données.
Cependant, il faut minimiser les transferts de mémoire entre le CPU et le GPU pour éviter des pertes de performance.



